# connexion sur la plate-forme

* premiere étape : 
 * se connecter sur formation 
```bash
ssh formation
```
 * ajouter en locale (sur votre machine dans le `.ssh/config`
```bash
Host formation
ForwardAgent yes
ForwardX11 yes
User cytec-marcf 
ProxyCommand ssh -T -q -o "ForwardAgent Yes" cytech-marcf@formation.plafrim.fr 'ssh-add -t 1 && nc plafrim 22'
```
 * Pour editer vos fichiers `vim` ou `emacs -nw`
```bash
emacs -nw mon_fichier.txt
```


# SLURM : mode interactif

* noeuds et partitions disponibles

```bash
[fuentes@miriel045 ~]$ sinfo
PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
sirocco      up   10:00:00      3   idle sirocco[03-04,06]
miriel       up 3-00:00:00      3  down* miriel[048,078,086]
miriel       up 3-00:00:00     13   idle miriel[050-052,057,063-064,067,069,079,084-085,087-088]
```
* allocation et utilisation interactive

```bash
[fuentes@miriel045 ~]$ salloc -p sirocco -N 1 --time=0:30:0 -w sirocco03
salloc: Granted job allocation 445861
salloc: Waiting for resource configuration
salloc: Nodes sirocco03 are ready for job
[fuentes@miriel045 ~]$ ssh sirocco03
```
* voir ses tâches

```bash
[fuentes@miriel045 ~]$ squeue -u fuentes
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            445861   sirocco     bash  fuentes  R       1:12      1 sirocco03
```

* supprimer une tâche

```bash
[fuentes@miriel045 ~]$ scancel 445861
```


# Modules

* des applications précompilées sont disponibles grâce au langage Tcl

```bash
[fuentes@miriel045 ~]$ module avail 2>&1 | grep compiler/cuda
compiler/cuda/10.0           mpi/openmpi/4.0.1-intel
compiler/cuda/10.2           mpi/openmpi/4.0.2
compiler/cuda/11.2           mpi/openmpi/4.0.2-testing
compiler/cuda/11.3           mpi/openmpi/4.0.3
compiler/cuda/11.4           mpi/openmpi/4.0.3-mlx
```

* chargement d'un module 

```bash
[fuentes@miriel045 ~]$ module load compiler/cuda/10.2
[fuentes@miriel045 ~]$ module list
Currently Loaded Modulefiles:
  1) compiler/cuda/10.2
[fuentes@miriel045 ~]$ which nvcc
/usr/local/cuda-10.2/bin/nvcc
```

* virer tous les modules `module purge` 

* liste les modules charger `module list`

* info sur un module `module show <mymodule>`


* environnement de travail : rajouter dans le fichier ~/envNew

```bash
module purge
module load compiler/cuda/11.4
module load compiler/gcc/9.3.0
module load build/cmake/3.21.3
```
* et rajouter `. ~/envNew` dans son .bashrc


# SLURM : soumission de tâches par lots
    
* on peut lancer des tâches par lots (batch)
    
* on écrit un fichier `runOn3.slurm`, par exemple
    
```bash
# #!/bin/sh 
#SBATCH --time=00:05:00
#SBATCH -w sirocco03
#SBATCH -p sirocco
#SBATCH -N 1
. ~/envNew
$*
```
* que l'on lance ensuite par `sbatch runOn3 <executable>`

```bash
[fuentes@miriel045 cuda_expe]$ sbatch runOn3.slurm C/1D/structTest && squeue -u fuentes
Submitted batch job 445863
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            445863   sirocco runOn3.s  fuentes PD       0:00      1 (None)
[fuentes@miriel045 cuda_expe]$ cat s
slurm-445862.out  slurm-445863.out  
[fuentes@miriel045 cuda_expe]$ cat slurm-445862.out 
data size (B) : 32
blocksize (B) : 1024
Good Kernel time : 0.0088ms
Bad Kernel time : 0.0084ms
```

# premier programme

* écrire un programme «bonjour monde»

```C
#include <cstdio>
__global__ void bonjour()
{
  printf("bonjour de la part de %d sur le GPU!\n", threadIdx.x);
}
int main()
{
  bonjour<<<1, 8>>>();
  cudaDeviceSynchronize();
  return 0;
}
```
* le compiler et le lancer sur le GPU

```bash
[fuentes@sirocco03 ~]$ nvcc -arch=sm_35 egunon.cu  -Wno-deprecated-gpu-targets && ./a.out
bonjour de la part de 0 sur le GPU!
bonjour de la part de 1 sur le GPU!
bonjour de la part de 2 sur le GPU!
bonjour de la part de 3 sur le GPU!
bonjour de la part de 4 sur le GPU!
bonjour de la part de 5 sur le GPU!
bonjour de la part de 6 sur le GPU!
bonjour de la part de 7 sur le GPU!
```


# introspection du GPU
    
* copier l'archive 
```bash
# cp /home/fuentes/deviceQuery.tgz .
```    
    
    
* decompresser l'archive du programme `deviceQuery` sur la frontale
```bash
tar xvzf deviceQuery.tgz  && cd deviceQuery && make
```

* executer deviceQuery par lots ou en mode interactif    
```bash
./deviceQuery
```

# Exemple de noyau élémentaire : homothétie

* coder le noyau homothetie

* coder une version CPU 

* comparer les 2 versions et verifier avec un assert que le calcul sur GPU est correct

* implanter une version retournée pour un pas de 2 du noyau

* comparer les perfs avec les fonctions de l'API

* comparer les perfs avec nvprof




# Array of Structs ou Struct of Arrays : c'est la question 😀

* on va coder nos particules de 2 façons
  * un tableau de structures
```C
struct AOS
{
  float x, y, z;
  float dx, dy, dz;
};
```
  * ou une structure de tableau
```C
struct SOA
{
  float * x, *y, *z;
  float * dx, *dy, *dz;
};
```
* l'algorithme va devoir faire $ x <- x + dx $, $y <- y + dy $, $z <- z + dz$ 

* coder une version utilisant AOS

* coder une version utilisant SOA

* s'assurer sur des petit cas qu'on calcule la meme chose.

* comparer les perfs


# Optimization 
## bande passante
- copier et décompresser `BP.tgz` et compiler le code (attention au module cuda)
```bash
cp /home/fuentes/BP.tgz . && tar xvzf BP.tgz && cd bandePassante && make
```
- executer les 2 codes avec le profileur 
```bash
nvprof ./pratiqueFM 
nvprof ./pratique
```
- relever les temps de chaque noyau et retrouver les résultats de BP réelle données en cours pour le K40M

$$ \frac{\#R_{ker} + \#W_{ker}}{10^9t} $$
 - Attention le nombre d'écriture et de lecture est en octets!

- à comparer avec la bande passante max théorique
```bash
./theo
```

- décommenter l'option de compilation `--ptxas-options=-v` dans le makefile et recompiler
```bash
make 2>&1 | c++filt
```
- Que vous dit le compilo ?

## Transferts asynchrones
### Mémoire verrouillée
- copier et décompresser async.tgz
```bash
cd ..
cp /home/fuentes/async.tgz . && tar xvzf async.tgz && cd async
```
- compiler
```
make 
```
- profiler `./pinned.x` avec une trace
```
nvprof --print-gpu-trace ./pinned.x
```
- dans la sortie, pretez attention aux colonnes `SrcMemType`, `Troughut` et `Duration`

- Qu'en deduisez vous ?

- optionnel (pour les courageux(ses)) : 💪
    - essayer de changer l'enum `cudaHostAllocDefault` par `cudaHostAllocMapped`
    - supprimer les transferts de l'hôte vers GPU (les 2 `cudaMemcpy`)
    - ecrire un noyau bidon (scale ou increment)
    - et verifier que la mémoire sur l'hôte est bien modifiée par le noyau

### transferts asynchrones
- lancer le programme `exAsync` et verifier que la version asynchrone est plus courte
```
./exAsync
```

- profiler le programme `exAsync.x`
```bash
nvprof --print-gpu-trace ./exAsync.x
```
- reconnaissez vous les differents appels du noyau ? notamment l'appel asynchrone en 4 morceaux

- changer le nombre de morceaux dans le sources et recompiler le programme, est plus rapide ou plus lent ?

#### note pour un affichage graphique
- Quand on dispose de `nvvp` en local (installer cuda sur sa machine). On peut extraire une trace 
```bash
nvprof --output-profile async.prof ./exAsync.x
```
- rapatrier la trace avec un scp et l'ouvrir avec nvvp
```
nvvp async.prof
```

## mémoire partagée : exemple de la transposition

- copier et decompresser les sources de la tranpostion
```bash
cd ..
cp /home/fuentes/transpose.tar . && tar xvf transpose.tar && cd tranposeSansSol
```

- compiler et faire tourner le programme
```
make
./transpose.x data/vinhamala.pgm
```
- charger le module feh et voir l'image transposée
```bash
module load visu/feh
feh --zoom 80 data/vinhamalaTrans.pgm
```
- écrire le noyau `transposeShared`, recompiler et l'executer
```bash
make
./transpose.x data/vinhamala.pgm 1
```
- verifier que le temps est inférieure à celui du noyau naif
- tester sans la synchronisation et voir que l'image produite est toute brouillée

### bonus (noyau sans conflits de bancs mémoire)
- copier le code du noyau `tranposeShared` dans `transposeNoConflicts` sans effacer la premier ligne avec `TILE_DIM+1`
- recompiler et utiliser le noyau
```bash
make
./transpose.x data/vinhamala.pgm 2
```
- verifier qu'il est encore plus rapide que `transposeShared`

- on peut aussi tester avec l'image `data/che.pgm` qui est encore plus grande